In [26]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from category_encoders import TargetEncoder

In [12]:
file_path = '../datasets/cleanedfix.csv'
df = pd.read_csv(file_path)

In [57]:
df['Casualty_severity'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 12316 entries, 0 to 12315
Series name: Casualty_severity
Non-Null Count  Dtype 
--------------  ----- 
12316 non-null  object
dtypes: object(1)
memory usage: 96.3+ KB


In [60]:
df['Day'].value_counts()

Day
Friday       2041
Thursday     1851
Wednesday    1840
Tuesday      1770
Monday       1681
Saturday     1666
Sunday       1467
Name: count, dtype: int64

# Label Encoding

In [13]:
df_label = df.copy()

In [14]:
label_encoders = {}
for col in df_label.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_label[col] = le.fit_transform(df_label[col])
    label_encoders[col] = le

print(df_label.head())


   Time  Age_band_of_driver  Sex_of_driver  Vehicle_driver_relation  \
0    17                   0              1                        0   
1    17                   1              1                        0   
2    17                   0              1                        0   
3     1                   0              1                        0   
4     1                   0              1                        0   

   Driving_experience  Type_of_vehicle  Owner_of_vehicle  \
0                   0                3                 3   
1                   3                4                 3   
2                   0                0                 3   
3                   2                4                 0   
4                   1                3                 3   

   Area_accident_occured  Lanes_or_Medians  Road_allignment  ...  \
0                      9                 2                5  ...   
1                      6                 4                5  ...   
2       

# One-Hot Encoding

In [15]:
df_ohe = pd.read_csv(file_path)

In [16]:
df_one_hot = pd.get_dummies(df_ohe.select_dtypes(include=['object']), dtype=int)

print(df_one_hot.head())

   Time_00:00:00  Time_01:00:00  Time_02:00:00  Time_03:00:00  Time_04:00:00  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              1              0              0              0   
4              0              1              0              0              0   

   Time_05:00:00  Time_06:00:00  Time_07:00:00  Time_08:00:00  Time_09:00:00  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   ...  Accident_severity_Fatal injury

# Target Encoding

Encoding target class menggunakan label encoding

In [36]:
# Menentukan kembali target dan fitur kategorikal
target_col = "Accident_severity"
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove(target_col)  # Menghapus target dari fitur kategorikal

# Mengubah nilai target menjadi numerik jika belum
df[target_col], _ = pd.factorize(df[target_col])

# Melakukan target encoding secara manual
df_encoded = df.copy()

for col in categorical_features:
    # Menghitung rata-rata target untuk setiap kategori
    target_mean = df.groupby(col)[target_col].mean()
    # Mengganti kategori dengan nilai target encoding
    df_encoded[col] = df[col].map(target_mean)

# Menampilkan hasil setelah encoding
df_encoded.head()


,Time,Age_band_of_driver,Sex_of_driver,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Area_accident_occured,Lanes_or_Medians,Road_allignment,...,encoded_Road_surface_type,encoded_Road_surface_conditions,encoded_Light_conditions,encoded_Weather_conditions,encoded_Type_of_collision,encoded_Vehicle_movement,encoded_Casualty_severity,encoded_Cause_of_accident,encoded_Accident_severity,encoded_Day
0,0.142508,0.170452,0.168313,0.165295,0.153759,0.167364,0.168997,0.193204,0.169308,0.167154,...,0,0,0,0,0,0,0,0,0,0
1,0.142508,0.158796,0.168313,0.165295,0.168435,0.168985,0.168997,0.160243,0.170706,0.167154,...,0,0,0,0,0,0,0,0,0,0
2,0.142508,0.170452,0.168313,0.165295,0.153759,0.173935,0.168997,0.168196,0.172289,0.167154,...,0,0,0,0,0,0,0,0,0,0
3,0.186567,0.170452,0.168313,0.165295,0.165315,0.168985,0.156580,0.160243,0.172289,0.161677,...,0,0,0,0,0,0,0,0,0,0
4,0.186567,0.170452,0.168313,0.165295,0.181018,0.167364,0.168997,0.157895,0.172289,0.167154,...,0,0,0,0,0,0,0,0,0,0


Encoding semua fitur menggunakan target encoding

# K-Fold Target Encoding

In [35]:
df_k = df.copy()
x = df_k.select_dtypes(include='object')
y = df_target_class['Accident_severity'].copy()

# Parameters
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize new columns for target encoding
for cat in x:
    df[f'encoded_{cat}'] = 0

# K-Fold Target Encoding for multiple categorical features
for train_index, val_index in kf.split(df):
    train, val = df.iloc[train_index], df.iloc[val_index]
    
    for cat in x:
        # Calculate the mean target for each category in the training set
        mean_target = train.groupby(cat)['Accident_severity'].mean()
        
        # Map the mean target to the validation set
        df.loc[val_index, f'encoded_{cat}'] = val[cat].map(mean_target)

# Display the DataFrame with encoded values
print(df)

TypeError: agg function failed [how->mean,dtype->object]